In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install tqdm transformers torch
import pandas as pd
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [3]:
df = pd.read_csv('/kaggle/input/samsungrecommend/RecommenderSamsungDevice - Sheet1.csv')
df.head()
df['Query'].value_counts()
possible_labels = df.Query.unique()

In [4]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Phone': 0,
 'Tab': 1,
 'TV': 2,
 'AC': 3,
 'Wash': 4,
 'Fridge': 5,
 'Vacuum': 6,
 'Dish': 7,
 'Micro': 8,
 'Watch': 9}

In [5]:
df['label'] = df.Query.replace(label_dict)


from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Query', 'label', 'data_type']).count()

/tmp/ipykernel_34/4262829081.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.Query.replace(label_dict)


Input
Query  label data_type       
AC     3     train         85
             val           15
Dish   7     train         85
             val           15
Fridge 5     train         85
             val           15
Micro  8     train         85
             val           15
Phone  0     train         85
             val           15
TV     2     train         85
             val           15
Tab    1     train         85
             val           15
Vacuum 6     train         85
             val           15
Wash   4     train         85
             val           15
Watch  9     train         85
             val           15

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Input.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [7]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)
from transformers import AdamW, get_linear_schedule_with_warmup

In [9]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')
import numpy as np

In [11]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [12]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [13]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/kaggle/working/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.01882696823335
Validation loss: 1.2371776461601258
F1 Score (Weighted): 0.8592444390998341


Epoch 2:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.7884056737620226
Validation loss: 0.3448184570670128
F1 Score (Weighted): 0.9536438755904828


...


Epoch 3
Training loss: 0.27508006706623966
Validation loss: 0.21254636481404304
F1 Score (Weighted): 0.9675292185839853


Epoch 4:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.14525881774303778
Validation loss: 0.18516794476658105
F1 Score (Weighted): 0.9669772089238161


Epoch 5:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.10277595595968231
Validation loss: 0.18174629237502812
F1 Score (Weighted): 0.9667939244663383


In [14]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('/kaggle/working/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)





Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Class: Phone
Accuracy: 12/15

Class: Tab
Accuracy: 15/15

Class: TV
Accuracy: 15/15

Class: AC
Accuracy: 15/15

Class: Wash
Accuracy: 14/15

Class: Fridge
Accuracy: 13/15

Class: Vacuum
Accuracy: 14/15

Class: Dish
Accuracy: 13/15

Class: Micro
Accuracy: 5/15

Class: Watch
Accuracy: 14/15



In [15]:
model.eval()
# Your input text
input_text = "how to fix my washing machine?"

# Tokenize and encode the input text
inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Make predictions
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=attention_mask)

# Extract predicted probabilities or class labels
predicted_probabilities = torch.softmax(output.logits, dim=1).cpu().numpy()
predicted_class = np.argmax(predicted_probabilities, axis=-1)

# Print the results
print("Predicted probabilities:", predicted_probabilities)
print("Predicted class:", predicted_class)
key_list = list(label_dict.keys())
val_list = list(label_dict.values())
 
# print key with val 100
position = val_list.index(predicted_class)
print(key_list[position])
key_list


Predicted probabilities: [[0.05097049 0.02972494 0.04694759 0.05964031 0.5591765  0.0631689
  0.04463558 0.07176366 0.02649357 0.04747843]]
Predicted class: [4]
Wash


['Phone',
 'Tab',
 'TV',
 'AC',
 'Wash',
 'Fridge',
 'Vacuum',
 'Dish',
 'Micro',
 'Watch']

In [16]:
import random

test_list = possible_labels

# printing original list
print("Original list is : " + str(test_list))


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
y=[]
for i in range(10000):
  rand_idx = random.randrange(len(test_list))
  random_label = test_list[rand_idx]
  y.append(random_label)
y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y
time=[]
for i in range(10000):
  time.append(random.randint(0, 23))
userid=[]
for i in range(10000):
  userid.append(random.randint(0, 60))
df=pd.DataFrame({'userid': userid, 'time': time, 'query': y})


df.shape


Original list is : ['Phone' 'Tab' 'TV' 'AC' 'Wash' 'Fridge' 'Vacuum' 'Dish' 'Micro' 'Watch']


(10000, 3)

In [25]:
reward=[]
for i in range(10000):
  reward.append(random.choice([0,1]))
df['reward']=reward

In [26]:
print(df)

      userid  time  query  reward
0         38     5      3       1
1         17     5      7       1
2         49    21      8       0
3         46    15      2       0
4         30    19      8       0
...      ...   ...    ...     ...
9995      55    12      1       0
9996      45    17      0       1
9997       0     0      1       0
9998       5     8      7       1
9999      15     1      0       1

[10000 rows x 4 columns]


In [19]:
df['userid']

userid=int(input())
import numpy as np
unique, frequency = np.unique(df['query'][df['userid']==userid], return_counts = True)
unique
frequency
unique
# Sort indices based on counts  in descending order
sorted_indices = np.argsort(frequency)[::-1]

# Get the top two labels and their frequencies
top_labels = unique[sorted_indices[:2]]
top_counts = frequency[sorted_indices[:2]]
top_labels
top_labeleng=le.inverse_transform(top_labels)
top_labeleng

 4


array(['Wash', 'AC'], dtype='<U6')

In [20]:
import random
import numpy as np

In [21]:
class ContextualLearner:
  # define the initial parameters when initializing class
  # parameters:
  #     learnerclass = SGDClassifier (preferred) or SGDRegressor
  #     rew_vec = array of possible rewards  for SGDClassifier and None for SGDRegressor
  def __init__(self, learnerclass, rew_vec):
    self.sgd = None
    self.hist = 50
    self.arm_sgd = {}
    self.dataX = {}
    self.dataY = {}
    self.rew_vec = rew_vec
    self.Learner = learnerclass

  # learn from an individual datapoint
  # parameters:
  #     ctx_vector = vector of context values pre normalized
  #     arm = action selected as a string
  #     reward = scalar reward value
  # returns:
  #     status = True or False if learning was successful
  def train(self, ctx_vector, arm, reward):
    X = []
    Y = []
    if ctx_vector is None or arm is None or reward is None:
      return False
    # if the arm classifier doesn't exist
    if arm not in self.arm_sgd.keys():
      self.arm_sgd[arm] = self.Learner()
      self.dataX[arm] = []
      self.dataY[arm] = []
    # get arm classifier and make prediction
    self.sgd = self.arm_sgd[arm]
    if len(self.dataX[arm]) > self.hist:
      X = self.dataX[arm][:-self.hist]
      Y = self.dataY[arm][:-self.hist]
    X.append(ctx_vector)
    X = np.asarray(X) #.reshape(1, -1)
    Y.append(reward) #= [reward]
    # fit the data point
    if self.rew_vec is not None:
      self.sgd.partial_fit(X, Y, self.rew_vec)
    else:
      self.sgd.partial_fit(X, Y)
    # add to data vectors
    self.dataX[arm].append(ctx_vector)
    self.dataY[arm].append(reward)
    return True

  # predict reward for an individual datapoint
  # parameters:
  #     ctx_vector = vector of context values pre normalized
  #     arm = action selected as a string
  # returns:
  #     reward = scalar reward value
  def predict(self, ctx_vector, arm):
    if ctx_vector is None or arm is None:
      return None
    # if the arm classifier doesn't exist
    if arm in self.arm_sgd.keys():
      # get arm classifier and make prediction
      self.sgd = self.arm_sgd[arm]
      X = ctx_vector
      X = np.asarray(X).reshape(1, -1)
      return self.sgd.predict(X)[0]
    # if nothing found return
    return 0

In [23]:
print(df)

      userid  time  query  reward
0         38     5      3       0
1         17     5      7       0
2         49    21      8       0
3         46    15      2       0
4         30    19      8       0
...      ...   ...    ...     ...
9995      55    12      1       0
9996      45    17      0       0
9997       0     0      1       1
9998       5     8      7       1
9999      15     1      0       1

[10000 rows x 4 columns]


In [27]:
from sklearn.linear_model import SGDRegressor, SGDClassifier

# Assuming df is your DataFrame containing user data

# count unique users
unique_users = df['userid'].unique()

# Define context vector fields
context_vector = ['time']

# Dictionary to hold bandit learners for each user
user_bandits = {}

# Loop over unique users
for user_id in unique_users:
    # Filter DataFrame for current user
    user_df = df[df['userid'] == user_id]
    
    # Initialize a new bandit learner for the current user
    user_bandits[user_id] = ContextualLearner(SGDRegressor, None)
    
    # Loop over records for the current user
    for index, record in user_df.iterrows():
        # Get context vector
        ctx_vec = record[context_vector].tolist()
        # Simple normalization for age
        ctx_vec[0] = ctx_vec[0] / 100.
        # Get recommendation - action or arm
        arm = record['query']
        # Get reward scalar value - rating given by user
        rew = record['reward']
        # Predict reward
        rew_pred = user_bandits[user_id].predict(ctx_vec, arm)
        # Train the bandit learner
        user_bandits[user_id].train(ctx_vec, arm, rew)

# Now, you can use the trained bandit learners for each user as needed
# For example, to predict rewards for a specific time for each user:
time = int(input("Enter time: "))
reward_lists = {}  # Dictionary to hold reward lists for each user

for user_id, bandit in user_bandits.items():
    # Assuming df is your DataFrame containing user data
    user_df = df[df['userid'] == user_id]
    
    # Initialize reward list for the current user
    reward_list = []
    
    # Loop over records for the current user
    for index, record in user_df.iterrows():
        # Get context vector
        ctx_vec = record[context_vector].tolist()
        # Simple normalization for age
        ctx_vec[0] = ctx_vec[0] / 100.
        # Get recommendation - action or arm
        arm = record['query']
        # Predict reward
        rew_pred = bandit.predict(ctx_vec, arm)
        reward_list.append(rew_pred)
    
    # Store the reward list for the current user
    reward_lists[user_id] = reward_list

# Now reward_lists dictionary contains reward lists for each user



Enter time:  3


In [30]:
def get_top_labels(user_id, num_labels=2):
    # Check if the user ID exists in the bandit learners dictionary
    if user_id in user_bandits:
        # Get the bandit learner for the specified user
        bandit = user_bandits[user_id]
        
        # Predict rewards for all possible actions
        rewards = {}
        num_actions = df['query'].nunique()

        for action in range(num_actions):  # Assuming num_actions is defined somewhere
            # Create a context vector (here, using default values, modify as needed)
            ctx_vec = [0.5]  # Placeholder context vector
            
            # Predict reward for the current action
            rew_pred = bandit.predict(ctx_vec, action)
            rewards[action] = rew_pred
        
        # Sort actions based on predicted rewards
        sorted_actions = sorted(rewards, key=rewards.get, reverse=True)
        
        # Get the top labels and their predicted rewards
        top_labels = [sorted_actions[i] for i in range(min(num_labels, len(sorted_actions)))]
        top_rewards = [rewards[action] for action in top_labels]
        
        return top_labels, top_rewards
    else:
        print("User ID not found.")
        return None, None

# Example usage: taking user ID input and getting top labels
user_id = int(input("Enter user ID: "))
top_labels, top_rewards = get_top_labels(user_id)

if top_labels is not None:
    print("Top labels for user {}: {}".format(user_id, top_labels))
    print("Corresponding rewards:", top_rewards)

Enter user ID:  4


Top labels for user 4: [5, 0]
Corresponding rewards: [0.06935503295674866, 0.06444238003781401]


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(df, test_size = 0.30, random_state = 42)

print(X_train.shape)
print(X_test.shape)


user_data = X_train.pivot_table(index = 'userid', columns = 'query', values = 'time', aggfunc='mean').fillna(0)
user_data.head()


# make a copy of train and test datasets
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['time'] = dummy_train['time'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['time'] = dummy_test['time'].apply(lambda x: 1 if x > 0 else 0)




# The movies not rated by user is marked as 1 for prediction
dummy_train = dummy_train.pivot_table(index = 'userid', columns = 'query', values = 'time', aggfunc='mean').fillna(1)

# The movies not rated by user is marked as 0 for evaluation
dummy_test = dummy_test.pivot_table(index ='userid', columns = 'query', values = 'time', aggfunc='mean').fillna(0)


dummy_train.head()
dummy_test.head()
from sklearn.metrics.pairwise import cosine_similarity

# User Similarity Matrix using Cosine similarity as a similarity measure between Users
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)


user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings




# np.multiply for cell-by-cell multiplication

user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()




user_final_ratings.iloc[42].sort_values(ascending = False)[0:5]



(7000, 4)
(3000, 4)
[[1.         0.95320026 0.95348032 ... 0.9633914  0.90903211 0.97131143]
 [0.95320026 1.         0.95894915 ... 0.97724512 0.93504682 0.98445468]
 [0.95348032 0.95894915 1.         ... 0.97180626 0.93146521 0.97841628]
 ...
 [0.9633914  0.97724512 0.97180626 ... 1.         0.9639072  0.97513513]
 [0.90903211 0.93504682 0.93146521 ... 0.9639072  1.         0.93540072]
 [0.97131143 0.98445468 0.97841628 ... 0.97513513 0.93540072 1.        ]]
(61, 61)


query
3    59.250602
7    43.651413
1     0.000000
0     0.000000
4     0.000000
Name: 42, dtype: float64